In [2]:
import pandas as pd
import numpy as np
import json
import os
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


In [3]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

In [5]:
categories = ['Patient History', 'Patient Demographics', 'Primary Diagnoses', 'Pre-admit Diagnoses',
       'Post-admit Diagnoses', 'Intervention', 'Allergies', 'Post-admit Diagnoses',
       'Discharge Location (to)', 'Medications during treatment', 'Medications on Discharge',
       'Post-discharge info (follow-ups, investigations, etc)', 'Treatment', 'Death Status']

In [3]:
df = pd.read_csv("/home/sael/EHR/physionet.org/patient_info.csv")

/tmp/ipykernel_1220410/4054733321.py:1: DtypeWarning: Columns (0,4,8,11,13,15,16,18,19,20,23,24,25,27,28,30,31,33,35,36,37,38,40,41,42,43,44,45,46,47,48,49,51,52,53,54,56,57,58,59,60,63,64,66,67,68,70,71,72,73,75,76,77,79,86,87,89,90,92,93,94,95,96,97,98,99,102,103,104,105,107,109,110,111,112,113,114,115,116,117,118,120,121,122,123,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,142,143,144,145,146,147,148,149,151,152,153,154,155,156,157,158,159,160,161,163,165,166,167,168,169,171,172,174,175,176,177,179) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/home/sael/EHR/physionet.org/patient_info.csv")


In [4]:
df.iloc[0, 1] = "Unique ID"

In [7]:
df.head()

,Category,Patient_ID,Patient History,Patient History.1,Patient History.2,Patient History.3,Patient History.4,Patient History.5,Patient History.6,Patient Demographics,...,Treatment.69,Treatment.70,Treatment.71,Treatment.72,Treatment.73,Treatment.74,Treatment.75,Death Status,Death Status.1,Death Status.2
0,Column Name,Unique ID,admittime,admission_location,enter_provider_id,admit_provider_id,insurance,admission_type,charttime,anchor_year_group,...,admit_provider_id,orderid,dilution_comparison,order_status,amount,micro_specimen_id,ordercomponenttypedescription,deathtime,dod,hospital_expire_flag
1,0,10000032_22841357,2180-06-26 18:27:00,EMERGENCY ROOM,NaN,P09Q6Y,Medicaid,EW EMER.,NaN,2014 - 2016,...,P09Q6Y,NaN,NaN,Inactive,NaN,NaN,NaN,NaN,2180-09-09,0
2,1,10000032_22841357,2180-06-26 18:27:00,EMERGENCY ROOM,NaN,P09Q6Y,Medicaid,EW EMER.,NaN,2014 - 2016,...,P09Q6Y,NaN,NaN,Inactive,NaN,NaN,NaN,NaN,2180-09-09,0
3,2,10000032_22841357,2180-06-26 18:27:00,EMERGENCY ROOM,NaN,P09Q6Y,Medicaid,EW EMER.,NaN,2014 - 2016,...,P09Q6Y,NaN,NaN,Inactive,NaN,NaN,NaN,NaN,2180-09-09,0
4,3,10000032_22841357,2180-06-26 18:27:00,EMERGENCY ROOM,NaN,P09Q6Y,Medicaid,EW EMER.,NaN,2014 - 2016,...,P09Q6Y,NaN,NaN,Inactive,NaN,NaN,NaN,NaN,2180-09-09,0


In [6]:
def rectify(df):
  new_cols = df.iloc[0]
  orig_cols = [col.split('.')[0] for col in df.columns]
  df.drop(df.index[0], inplace = True)
  df.columns = pd.MultiIndex.from_tuples([col1, col2] for col1, col2 in zip(orig_cols, new_cols))
  return df
df = rectify(df)

# **Creating the dictionary for indexing**

In [7]:
indexed_dict = {}
for row in range(len(df)):
  indexed_dict[df.iloc[row, 1]] = df.iloc[row, 2:]

In [8]:
test_id = "10000032_22841357"

In [12]:
indexed_dict[test_id].keys().get_level_values(0).unique()

Index(['Patient History', 'Patient Demographics', 'Primary Diagnoses',
       'Intervention', 'Discharge Location (to)',
       'Medications during treatment',
       'Post-discharge info (follow-ups, investigations, etc)', 'Treatment',
       'Death Status'],
      dtype='object')

# **Retrieval Function**

In [9]:
def retrieval(id, indexed_dict):
  if id in indexed_dict:
    return indexed_dict[id]
  else:
    print("Entered ID doesn't exist in the records")
    return None

# **Augmentation step**

In [18]:
test = indexed_dict[test_id].get("Patient History")
res = f'''For the patient id - {test_id}, note the following details from the Electronic Health Records:\n'''
for key, val in test.items():
    res += f'''The {key} is {val}. \n '''
print(res)

For the patient id - 10000032_22841357, note the following details from the Electronic Health Records:
The admittime is 2180-06-26 18:27:00. 
 The admission_location is EMERGENCY ROOM. 
 The enter_provider_id is nan. 
 The admit_provider_id is P09Q6Y. 
 The insurance is Medicaid. 
 The admission_type is EW EMER.. 
 The charttime is nan. 
 


Base Model

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
# model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B")
# tokenizer.pad_token_id = model.config.eos_token_id

Chat model

Llama 3 7B

In [16]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
tokenizer.pad_token_id = model.config.eos_token_id

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.20it/s]


In [ ]:
Mixtral 7 X 8 

In [13]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1")
tokenizer.pad_token_id = model.config.eos_token_id

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████| 19/19 [10:34<00:00, 33.37s/it]


Gemini pro

In [12]:
API_KEY = "YOUR_API_KEY"

In [14]:
import google.generativeai as genai
import google.ai.generativelanguage as glm
genai.configure(
    api_key = API_KEY)

In [15]:
print(genai.get_model('models/gemini-pro'),'\n')
print(genai.get_model('models/gemini-pro-vision'),'\n')

Model(name='models/gemini-pro',
      base_model_id='',
      version='001',
      display_name='Gemini 1.0 Pro',
      description='The best model for scaling across a wide range of tasks',
      input_token_limit=30720,
      output_token_limit=2048,
      supported_generation_methods=['generateContent', 'countTokens'],
      temperature=0.9,
      top_p=1.0,
      top_k=None) 

Model(name='models/gemini-pro-vision',
      base_model_id='',
      version='001',
      display_name='Gemini 1.0 Pro Vision',
      description='The best image understanding model to handle a broad range of applications',
      input_token_limit=12288,
      output_token_limit=4096,
      supported_generation_methods=['generateContent', 'countTokens'],
      temperature=0.4,
      top_p=1.0,
      top_k=32) 



In [16]:
model = genai.GenerativeModel(model_name="gemini-pro")

In [29]:
prompt = [
    "You are a medical AI agent tasked with generating a discharge summary based on the provided information about the patient.",
    "Your task is to generate a coherent passage summarizing all the following information effectively, ensuring all important details are mentioned to be included in the Dischary Summary for the patient.",
    f"{res}",
]
response = model.generate_content(prompt)
print(response.text)

**Discharge Summary**

**Patient:** 10000032_22841357

**Admission:**
* Admitted to the Emergency Room on 2180-06-26 at 18:27:00
* Admitted by: P09Q6Y
* Admission type: EW EMER.
* Insurance: Medicaid

**Additional Information:**
* No provider information is available for admission or discharge.
* No chart time is available.


Instruct prompt

In [42]:
test_id = "10000032_22841357"
# categories = ['Patient History', 'Patient Demographics', 'Treatment']
categories = ['Patient History', 'Patient Demographics', 'Primary Diagnoses', 'Pre-admit Diagnoses',
       'Post-admit Diagnoses', 'Intervention', 'Allergies', 'Post-admit Diagnoses',
       'Discharge Location (to)', 'Medications during treatment', 'Medications on Discharge',
       'Post-discharge info (follow-ups, investigations, etc)', 'Treatment', 'Death Status']

In [45]:
discharge_summary = ""

In [48]:
for cat in categories:
    # Getting information for the DST category
    category_info = indexed_dict[test_id].get(cat)
    info_para = f'''For the patient id - {test_id}, note the following details from the Electronic Health Records for the category - {cat}:\n'''
    if category_info is not None:
        for key, val in category_info.items():
            info_para += f'''The {key} is {val}. \n '''
        prompt = [
    "You are a medical AI agent tasked with generating a discharge summary based on the provided information about the patient.",
    "Your task is to generate a coherent passage summarizing all the following information effectively, ensuring all important details are mentioned to be included in the corresponding Dischary Summary category for the patient.",
    f"{info_para}",
                ]
        # print(prompt)
        # break
        generated_text = model.generate_content(prompt)
        if generated_text is not None:
            discharge_summary += f"{cat} : {generated_text.text}\n"
print(discharge_summary)

Patient History : **Patient History**

**Patient ID:** 10000032_22841357

**Admit Time:** 2180-06-26 18:27:00

**Admission Location:** EMERGENCY ROOM

**Enter Provider ID:** nan

**Admit Provider ID:** P09Q6Y

**Insurance:** Medicaid

**Admission Type:** EW EMER.

**Charttime:** nan
Patient Demographics : **Patient Demographics**

**Patient ID:** 10000032_22841357

**Subject ID:** 10000032

**Gender:** Female (F)

**Anchor Age:** 52 years

**Anchor Year:** 2180

**Anchor Year Group:** 2014 - 2016

**Race:** White

**Marital Status:** Widowed

**Language:** English

**Hospitalization ID:** 22841357
Primary Diagnoses : **Patient ID:** 10000032_22841357

**Primary Diagnosis:**

**ICD-9 Code:** 5491

**Description:** DISORDERS OF LIVER EXCEPT MALIG,CIRR,ALC HEPA W CC

**DRG Characteristics:**

**DRG Code:** 442

**DRG Type:** HCFA

**DRG Severity:** Not available

**DRG Mortality:** Not available
Intervention : **Intervention:**

- **Intervention Type:** Unit Dose
- **Administration Status

For HuggingFace Loaded models

In [21]:
def generate_discharge_summary(id, indexed_dict, categories, model, tokenizer):
  # Retrieving information for the input ID (subject_id + hadm_id)
  patient_info_dict = retrieval(id, indexed_dict)
  if patient_info_dict is None:
    return "Error: Patient ID not found in records."

  # Generating text for each category and then concatenating it one below the other
  discharge_summary = ""

  # Iterating through each category
  for cat in categories:
    # Getting information for the DST category
    category_info = patient_info_dict.get(cat)
    info_para = f'''For the patient id - {id}, note the following details from the Electronic Health Records for the category - {cat}:\n'''
    for key, val in category_info.items():
        info_para += f'''The {key} is {val}. \n '''

    if category_info is not None:

        messages = [
          {
            "role" : "system",
            "content" : "You are a medical AI agent tasked with generating a discharge summary based on the provided information about the patient."

          },
          {
            "role" : "user",
            "content" : f'''
            {info_para} \n
            Your task is to generate a coherent passage summarizing all the available information effectively, 
            ensuring all important details are mentioned to be included in the Dischary Summary for the patient.
        '''
          }
        ]
        inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model.generate(inputs, temperature=0.7, max_length=1000, repetition_penalty=1.9, do_sample=True)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=False)
        generated_text = generated_text.split("assistant<|end_header_id|>")[1]
        # Combining category title and the generated information
        discharge_summary += f"{cat} : {generated_text}\n"

  return discharge_summary

In [ ]:
cache = generate_discharge_summary(test_id, indexed_dict, categories, model)
print(cache)